In [6]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from plyer import notification  
from datetime import datetime
import json
import aiohttp
import asyncio

engine = create_engine('sqlite:///cnpj_database.db')

def alerta(nivel, etapa, cnpj=None):
    titulo_notificacao = {
        1: 'Sucesso - Alerta Baixo',
        2: 'Erro Grave - Alerta Médio',
        3: 'Erro Muito Grave - Alerta Alto'
    }.get(nivel, 'Alerta Desconhecido')

    data_hora_atual = datetime.now().strftime('%d-%m-%y %H:%M:%S')
    msg_notificacao = f"Etapa: {etapa}\nCNPJ: {cnpj}\n{data_hora_atual}"

    notification.notify(
        title=titulo_notificacao,
        message=msg_notificacao,
        app_name='Extrator de APIs',
        timeout=10
    )

def limpar_dados(df):
    df.drop(columns=[col for col in ['qsa'] if col in df.columns], inplace=True)
    df.rename(columns={'uf': 'UF', 'cep': 'CEP', 'cnpj': 'CNPJ'}, inplace=True)

    for col in df.columns:
        if df[col].dtype == 'object':
            if isinstance(df[col].iloc[0], (list, dict)):
                df[col] = df[col].apply(json.dumps)

    df.fillna('Não informado', inplace=True)
    return df

async def fetch_data(session, url, nome):
    async with session.get(url) as response:
        if response.status == 200:
            return await response.json()
        else:
            print(f"Erro ao acessar {nome}: {response.status}")
            alerta(2, f"Erro ao acessar a {nome}: {response.status}")
            return None

async def process_cnpj(cnpj):
    async with aiohttp.ClientSession() as session:
        tabelas = {}
        urls = {
            'Tabela1': f'https://brasilapi.com.br/api/cnpj/v1/{cnpj}'
        }

        dados_tabela1 = await fetch_data(session, urls['Tabela1'], 'Tabela1')
        if dados_tabela1:
            tabelas['Tabela1'] = pd.DataFrame([dados_tabela1])
            print("Tabela1 extraída com sucesso!")
            alerta(1, "Tabela1 extraída com sucesso.", cnpj)

            CEP = dados_tabela1.get('cep')

        
            urls['Tabela2'] = f'https://brasilapi.com.br/api/cep/v1/{CEP}'

        
            dados_tabela2 = await fetch_data(session, urls['Tabela2'], 'Tabela2')

            if dados_tabela2:
                tabelas['Tabela2'] = pd.DataFrame([dados_tabela2])
                print("Tabela2 extraída com sucesso!")
                alerta(1, "Tabela2 extraída com sucesso.", cnpj)

        
            for nome in tabelas:
                tabelas[nome] = limpar_dados(tabelas[nome])

        
            if 'Tabela1' in tabelas and 'Tabela2' in tabelas:
                try:
                    
                    tabela_final = pd.merge(tabelas['Tabela1'], tabelas['Tabela2'], on='CEP', how='inner')

                    
                    print(f"Tabela final para CNPJ {cnpj}:")
                    print(tabela_final.head())

                    
                    tabela_final.to_sql(f"Tabela_Final_{cnpj}", engine, if_exists='replace', index=False)
                    print(f"Tabela final {cnpj} salva no banco de dados.")
                except Exception as e:
                    print(f"Erro ao juntar ou salvar as tabelas: {e}")
                    alerta(3, f"Erro ao juntar ou salvar tabelas para o CNPJ {cnpj}", cnpj)
        else:
            print(f"Tabela1 não extraída para o CNPJ {cnpj}.")
            alerta(3, f"Tabelas incompletas para o CNPJ {cnpj}", cnpj)


async def main(cnpjs):
    tasks = [process_cnpj(cnpj) for cnpj in cnpjs]
    await asyncio.gather(*tasks)


cnpjs = ['30306294000145', '31620016000120']


await main(cnpjs)








Tabela1 extraída com sucesso!
Tabela1 extraída com sucesso!
Tabela2 extraída com sucesso!
Tabela final para CNPJ 30306294000145:
   UF       CEP            CNPJ           pais          email   porte  \
0  RJ  22250911  30306294000145  Não informado  Não informado  DEMAIS   

     bairro numero      ddd_fax       municipio  ...  \
0  BOTAFOGO  00501  02125149600  RIO DE JANEIRO  ...   

  qualificacao_do_responsavel  descricao_situacao_cadastral  \
0                          10                         ATIVA   

  descricao_tipo_de_logradouro descricao_motivo_situacao_cadastral  \
0                        PRAIA                          SEM MOTIVO   

   descricao_identificador_matriz_filial state            city  neighborhood  \
0                                 MATRIZ    RJ  Rio de Janeiro      Botafogo   

               street   service  
0  Praia Botafogo 501  open-cep  

[1 rows x 52 columns]
Tabela final 30306294000145 salva no banco de dados.
Tabela2 extraída com sucesso!
Tabela f